<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/SafeGraph-visit-trend-by-category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import gzip as gz
import os
from pathlib import Path
import json
import matplotlib.pyplot as plt

from glob import glob
import time

import os
from shutil import copyfile, copytree

## Load the SafeGraph data from Google Drive
Load visit patterns from march-june 2019 for the desired place types, keep places that have data for entire march-june range
* places
* cbg_pop
* patterns_2019

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
!ls drive/My\ Drive/SafeGraph

 census_vs_safegraph_IL.csv    places			 weekly
'Data Science Resources.url'  'Places Schema.url'	'Weekly Patterns.url'
 historic		       places_v3_safegraph.csv	 weekly_v2
 monthly		       social-distancing
 Patterns_NE-IL_2019.csv       vis_2020


In [72]:
# grab the SafeGraph core places
files = glob("drive/My Drive/SafeGraph/*.csv*")
print(files)

['drive/My Drive/SafeGraph/places_v3_safegraph.csv', 'drive/My Drive/SafeGraph/census_vs_safegraph_IL.csv', 'drive/My Drive/SafeGraph/Patterns_NE-IL_2019.csv.gz']


In [73]:
if not os.path.exists('sg_data'):
  os.mkdir('sg_data')

In [74]:
# copy these files from Google Drive to colab disk
for f in files:
  print(f)
  copyfile(f, 'sg_data/' + os.path.basename(f))

drive/My Drive/SafeGraph/places_v3_safegraph.csv
drive/My Drive/SafeGraph/census_vs_safegraph_IL.csv
drive/My Drive/SafeGraph/Patterns_NE-IL_2019.csv.gz


In [77]:
#!ls sg_data
local_files = glob("sg_data/*.csv*")
print(local_files)

['sg_data/places_v3_safegraph.csv', 'sg_data/Patterns_NE-IL_2019.csv.gz', 'sg_data/census_vs_safegraph_IL.csv']


In [78]:
place_df = pd.read_csv(local_files[0])
place_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74782 entries, 0 to 74781
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   numeric_id          74782 non-null  int64  
 1   place_id            74782 non-null  object 
 2   place_type          74782 non-null  object 
 3   latitude            74782 non-null  float64
 4   longitude           74782 non-null  float64
 5   name                74782 non-null  object 
 6   zipcode             74782 non-null  int64  
 7   visits_IL           64394 non-null  float64
 8   unique_visitors_IL  64394 non-null  float64
 9   cbg                 74782 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 5.7+ MB


In [79]:
place_types = [key for key in place_df['place_type'].value_counts().keys()]
place_types

['restaurant',
 'clinic',
 'recreation',
 'store',
 'worship',
 'pharmacy',
 'daycare',
 'supermarket',
 'convenience store',
 'long term care',
 'library',
 'bar',
 'museum',
 'hospital']

In [80]:
# Exrtact desired place types from v3 safegraph places file (7 county area)
places = place_df[['place_id', 'place_type']]

In [81]:
# Load cbg population vs safegraph devices
cbg_pop = pd.read_csv(local_files[2]) # https://anl.box.com/s/k2e7e833gfekqsihaachbj9pfsshhc2g
cbg_pop = cbg_pop[cbg_pop['state'] == 'IL'] # base table has a few non-IL cbgs
print("Number of IL cbgs (should be 9691): {}".format(len(cbg_pop)))

Number of IL cbgs (should be 9691): 9691


In [88]:
# load 2019 patterns
patterns_2019 = pd.read_csv(local_files[1]) # https://anl.box.com/s/zf3xt1cbd6vjs40e9njw91jm1jgkxj83

In [89]:
len(patterns_2019)

1233810

In [90]:
# keep patterns for the selected place types only
patterns_2019 = patterns_2019.merge(places, left_on = 'safegraph_place_id', right_on = 'place_id')

In [91]:
months = [3, 4, 5, 6, 7, 8] # march-august

# extract month from safegraph date range 
def get_month(date):
    ymd = date.split('-')
    day = int(ymd[2].split('T')[0])
    if day != 1:
        print('date range not starting with 1st of month')
    return int(ymd[1]) # month

In [92]:
patterns_2019['month'] = patterns_2019['date_range_start'].apply(get_month)
# keep only march-june patterns
patterns_2019 = patterns_2019[patterns_2019['month'].isin(months)]
#keep only for places that have data for entire march-june period
counts = patterns_2019.groupby('safegraph_place_id').count()
counts = counts[counts['month'] == len(months)]
patterns_2019_flt = patterns_2019[patterns_2019['safegraph_place_id'].isin(counts.index)]

In [93]:
len(patterns_2019_flt)

349962

# Do the same for march-june 2020 visit patterns

In [96]:
# monthly patterns file in the Google drive
print(glob("drive/My Drive/SafeGraph/vis_2020/*/*.csv.gz"))

['drive/My Drive/SafeGraph/vis_2020/08/patterns-part1.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/08/patterns-part2.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/08/patterns-part3.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/08/patterns-part4.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/07/patterns-part1.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/07/patterns-part2.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/07/patterns-part3.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/07/patterns-part4.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/06/patterns-part1.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/06/patterns-part2.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/06/patterns-part3.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/06/patterns-part4.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/05/patterns-part2.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/05/patterns-part1.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/05/patterns-part4.csv.gz', 'drive/My Drive/SafeGraph/vis_2020/05/patterns-part3.csv.gz', 'drive/

In [34]:
# copy these files from Google Drive to colab disk
copytree("drive/My Drive/SafeGraph/vis_2020", "vis_2020")

'vis_2020'

In [95]:
!ls

drive  sample_data  sg_data  vis_2020


In [106]:
# This code directly extracts data from raw .csv.gz patterns files, for the place types defined at the beginning
# Download files from here- https://catalog.safegraph.io/app/browse
#   From Monthly Patterns Jan 2018 - Apr 2020: download subfolders 2020/03 and 2020/04
#   From Montnly Patterns May 2020- Present: download everything (confusing folder structure, couldn't isolate may and june)
# Dump all the files in one folder and set the path accordingly on the line below
rootdir = Path('vis_2020')
# Return a list of regular files only, not directories
file_list = [f for f in rootdir.glob('**/*.csv.gz') if f.is_file()]
print(file_list)

[PosixPath('vis_2020/07/patterns-part2.csv.gz'), PosixPath('vis_2020/07/patterns-part3.csv.gz'), PosixPath('vis_2020/07/patterns-part1.csv.gz'), PosixPath('vis_2020/07/patterns-part4.csv.gz'), PosixPath('vis_2020/05/patterns-part2.csv.gz'), PosixPath('vis_2020/05/patterns-part3.csv.gz'), PosixPath('vis_2020/05/patterns-part1.csv.gz'), PosixPath('vis_2020/05/patterns-part4.csv.gz'), PosixPath('vis_2020/06/patterns-part2.csv.gz'), PosixPath('vis_2020/06/patterns-part3.csv.gz'), PosixPath('vis_2020/06/patterns-part1.csv.gz'), PosixPath('vis_2020/06/patterns-part4.csv.gz'), PosixPath('vis_2020/04/patterns-part2.csv.gz'), PosixPath('vis_2020/04/patterns-part3.csv.gz'), PosixPath('vis_2020/04/patterns-part1.csv.gz'), PosixPath('vis_2020/04/patterns-part4.csv.gz'), PosixPath('vis_2020/03/patterns-part2.csv.gz'), PosixPath('vis_2020/03/patterns-part3.csv.gz'), PosixPath('vis_2020/03/patterns-part1.csv.gz'), PosixPath('vis_2020/03/patterns-part4.csv.gz'), PosixPath('vis_2020/08/patterns-part2.c

In [107]:
patterns_2020 = pd.DataFrame()

for fl in file_list:
    print(fl)
    part = pd.read_csv(fl)
    
    # keep patterns for the selected place types only
    part = part.merge(places, left_on = 'safegraph_place_id', right_on = 'place_id')
    part['month'] = part['date_range_start'].apply(get_month)
    # keep only march-june patterns
    part = part[part['month'].isin(months)]

    patterns_2020 = patterns_2020.append(part)

vis_2020/07/patterns-part2.csv.gz
vis_2020/07/patterns-part3.csv.gz
vis_2020/07/patterns-part1.csv.gz
vis_2020/07/patterns-part4.csv.gz
vis_2020/05/patterns-part2.csv.gz
vis_2020/05/patterns-part3.csv.gz
vis_2020/05/patterns-part1.csv.gz
vis_2020/05/patterns-part4.csv.gz
vis_2020/06/patterns-part2.csv.gz
vis_2020/06/patterns-part3.csv.gz
vis_2020/06/patterns-part1.csv.gz
vis_2020/06/patterns-part4.csv.gz
vis_2020/04/patterns-part2.csv.gz
vis_2020/04/patterns-part3.csv.gz
vis_2020/04/patterns-part1.csv.gz
vis_2020/04/patterns-part4.csv.gz
vis_2020/03/patterns-part2.csv.gz
vis_2020/03/patterns-part3.csv.gz
vis_2020/03/patterns-part1.csv.gz
vis_2020/03/patterns-part4.csv.gz
vis_2020/08/patterns-part2.csv.gz
vis_2020/08/patterns-part3.csv.gz
vis_2020/08/patterns-part1.csv.gz
vis_2020/08/patterns-part4.csv.gz


In [109]:
#keep only for places that have data for entire march-june period
counts = patterns_2020.groupby('safegraph_place_id').count()
counts = counts[counts['month'] == len(months)]
patterns_2020_flt = patterns_2020[patterns_2020['safegraph_place_id'].isin(counts.index)]
patterns_2020_flt.to_csv('patterns_covid.csv.gz', compression='gzip')

In [110]:
len(patterns_2020_flt)

323298

In [111]:
copyfile('patterns_covid.csv.gz', 'drive/My Drive/SafeGraph/patterns_covid.csv.gz')

'drive/My Drive/SafeGraph/patterns_covid.csv.gz'

In [ ]:
place_types = ['restaurant', 'recreation', 'supermarket'] # place types for which to compare visit patterns
months = [3, 4, 5, 6] # march-june

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5284986 entries, 0 to 5284985
Data columns (total 18 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   safegraph_place_id         object 
 1   parent_safegraph_place_id  object 
 2   location_name              object 
 3   safegraph_brand_ids        object 
 4   brands                     object 
 5   top_category               object 
 6   sub_category               object 
 7   naics_code                 float64
 8   latitude                   float64
 9   longitude                  float64
 10  street_address             object 
 11  city                       object 
 12  region                     object 
 13  postal_code                int64  
 14  iso_country_code           object 
 15  phone_number               float64
 16  open_hours                 object 
 17  category_tags              object 
dtypes: float64(4), int64(1), object(13)
memory usage: 725.8+ MB


In [ ]:
place_df.head()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
0,sg:00201f75b90b47d3be813562f946d97f,NaN,First Presbyterian Church of Stamford,NaN,NaN,Religious Organizations,Religious Organizations,813110.0,41.062975,-73.538788,1101 Bedford St,Stamford,CT,6905,US,1.203325e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN
1,sg:002b991d388c42a3a64db6066f8c4c0e,NaN,El Gran Sabor,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,38.926747,-79.848400,413 Kerens Ave,Elkins,WV,26241,US,1.304637e+10,NaN,"Late Night,Latin American Food"
2,sg:002e6e7d2a0642a197e51a44bfce7a3e,NaN,Public Storage,SG_BRAND_1a2e7806afb9c1c7d60fd00219204dde,Public Storage,Lessors of Real Estate,Lessors of Miniwarehouses and Self-Storage Units,531130.0,33.035779,-96.699591,2801 Avenue K,Plano,TX,75074,US,NaN,NaN,NaN
3,sg:0045ce84ae2847a08abe59b4139f871e,NaN,TravelCenters of America,SG_BRAND_27a15966382ba3e78015b40d2e1fa976,TravelCenters of America,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,38.428077,-95.726311,2775 US Highway 75,Lebo,KS,66856,US,NaN,NaN,NaN
4,sg:00a3a6674d95443c80e481bdf68e84d2,sg:6449a1a66fcc451abc993849f61445f0,Great American Cookie Company,NaN,NaN,Bakeries and Tortilla Manufacturing,Retail Bakeries,311811.0,30.257337,-97.807172,Barton Creek Mall,Austin,TX,78701,US,1.512328e+10,NaN,NaN


In [ ]:
# translate latitude and longitude to geopandas
place_gdf = gpd.GeoDataFrame(
    place_df, geometry = gpd.points_from_xy(place_df.longitude, place_df.latitude),
    crs = "epsg:4326" # https://geopandas.readthedocs.io/en/v0.6.0/projections.html#coordinate-reference-systems
).to_crs(epsg=3528) # https://spatialreference.org/ref/epsg/3528/

In [ ]:
place_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5284986 entries, 0 to 5284985
Data columns (total 19 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   safegraph_place_id         object  
 1   parent_safegraph_place_id  object  
 2   location_name              object  
 3   safegraph_brand_ids        object  
 4   brands                     object  
 5   top_category               object  
 6   sub_category               object  
 7   naics_code                 float64 
 8   latitude                   float64 
 9   longitude                  float64 
 10  street_address             object  
 11  city                       object  
 12  region                     object  
 13  postal_code                int64   
 14  iso_country_code           object  
 15  phone_number               float64 
 16  open_hours                 object  
 17  category_tags              object  
 18  geometry                   geometry
dtypes: float64(4)

In [ ]:
# limit to Illinois (5.3M --> 200K places)
place_gdf_IL = place_gdf[place_gdf.region == 'IL']
place_gdf_IL.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 200614 entries, 5 to 5284973
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   safegraph_place_id         200614 non-null  object  
 1   parent_safegraph_place_id  24590 non-null   object  
 2   location_name              200614 non-null  object  
 3   safegraph_brand_ids        47267 non-null   object  
 4   brands                     47267 non-null   object  
 5   top_category               198532 non-null  object  
 6   sub_category               198532 non-null  object  
 7   naics_code                 198532 non-null  float64 
 8   latitude                   200614 non-null  float64 
 9   longitude                  200614 non-null  float64 
 10  street_address             200614 non-null  object  
 11  city                       200614 non-null  object  
 12  region                     200614 non-null  object  
 13  posta

In [ ]:
place_gdf_IL.crs

<Projected CRS: EPSG:3528>
Name: NAD83(NSRS2007) / Illinois East
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA - Illinois - SPCS - E
- bounds: (-89.28, 37.06, -87.02, 42.5)
Coordinate Operation:
- name: SPCS83 Illinois East zone (meters)
- method: Transverse Mercator
Datum: NAD83 (National Spatial Reference System 2007)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## Get the census block groups of NE Illinois

FIPS code for the counties
* Cook: 031
* Dupage: 043
* Grundy: 063
* Kane: 089
* Kankakee: 091
* Kendall: 093
* Lake: 097
* McHenry: 111
* Will: 197

In [ ]:
illinois_cbg = gpd.read_file('https://raw.githubusercontent.com/loganpowell/census-geojson/master/GeoJSON/500k/2019/17/block-group.json', 
                   crs = from_epsg(2163)).to_crs(epsg = 3528)

In [ ]:
# limit to 9 counties mentioned above
target_cbg = illinois_cbg[illinois_cbg.COUNTYFP.isin(['031', '043', '063', '089', '091', '093', '097', '111', '197'])]
target_cbg = target_cbg[['GEOID', 'geometry']]
target_cbg.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 5998 entries, 0 to 9688
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   GEOID     5998 non-null   object  
 1   geometry  5998 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 140.6+ KB


In [ ]:
print(target_cbg.crs)
#target_cbg.head()
print(place_gdf_IL.crs)

epsg:3528
epsg:3528


In [ ]:
place_gdf_target = gpd.sjoin(place_gdf_IL, target_cbg, how="inner", op="within").drop("index_right", axis = 1)
place_gdf_target.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 132091 entries, 43 to 5230499
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   safegraph_place_id         132091 non-null  object  
 1   parent_safegraph_place_id  18968 non-null   object  
 2   location_name              132091 non-null  object  
 3   safegraph_brand_ids        30605 non-null   object  
 4   brands                     30605 non-null   object  
 5   top_category               130284 non-null  object  
 6   sub_category               130284 non-null  object  
 7   naics_code                 130284 non-null  float64 
 8   latitude                   132091 non-null  float64 
 9   longitude                  132091 non-null  float64 
 10  street_address             132091 non-null  object  
 11  city                       132091 non-null  object  
 12  region                     132091 non-null  object  
 13  post

In [ ]:
place_gdf_target['naics_2digit'] = place_gdf_target['naics_2digit'].fillna(0.0).astype(int)
place_gdf_target.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 132091 entries, 43 to 5230499
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   safegraph_place_id         132091 non-null  object  
 1   parent_safegraph_place_id  18968 non-null   object  
 2   location_name              132091 non-null  object  
 3   safegraph_brand_ids        30605 non-null   object  
 4   brands                     30605 non-null   object  
 5   top_category               130284 non-null  object  
 6   sub_category               130284 non-null  object  
 7   naics_code                 130284 non-null  float64 
 8   latitude                   132091 non-null  float64 
 9   longitude                  132091 non-null  float64 
 10  street_address             132091 non-null  object  
 11  city                       132091 non-null  object  
 12  region                     132091 non-null  object  
 13  post

In [ ]:
naics_summary = place_gdf_target.naics_code.value_counts()
naics_summary.head(20)

722511.0    13331
621111.0     6470
812112.0     5309
621210.0     4653
813110.0     4193
722513.0     3958
712190.0     3918
524210.0     3847
713940.0     3787
611110.0     3428
722515.0     3415
522110.0     3215
811111.0     2834
624410.0     2694
447110.0     2415
445110.0     2232
621330.0     2016
812199.0     1939
812113.0     1706
621310.0     1557
Name: naics_code, dtype: int64

In [ ]:
naics_2digit_summary = place_gdf_target.naics_2digit.value_counts()
naics_2digit_summary

72    21924
44    21850
62    21568
81    20696
52     9647
71     9014
45     8074
61     4885
53     2871
54     2180
51     2122
0      1807
56     1043
92      940
31      853
49      791
48      665
42      535
33      237
23      169
32      153
22       59
11        6
6         2
Name: naics_2digit, dtype: int64

In [ ]:
cbg_summary = place_gdf_target.GEOID.value_counts()
cbg_summary.head(20)

170318391001    1237
170318046033     709
170313204001     505
170313201002     393
170313201001     381
170318015001     358
170438446011     322
170938901011     285
170318069001     284
170898526013     272
170310817002     264
170318330001     264
170318422001     260
170318391002     258
171118713064     256
170317707001     253
170310814011     251
170438465042     243
170318016011     237
170438465192     236
Name: GEOID, dtype: int64

## Generate the IL places file

In [ ]:
file_name = 'core_places_IL'
place_gdf_target.to_file(file_name + '.geojson', driver='GeoJSON')

In [ ]:
# also to a plain csv file
pd.DataFrame(place_gdf_target.drop(columns='geometry')).to_csv(file_name + '.csv')

In [ ]:
# copy to google Drive
copyfile(file_name + '.geojson', "drive/My Drive/SafeGraph/places/" + file_name + '.geojson')
copyfile(file_name + '.csv', "drive/My Drive/SafeGraph/places/" + file_name + '.csv')

'drive/My Drive/SafeGraph/places/core_places_IL.csv'